In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import linalg, optimize

In [2]:
nut_calc = pd.read_excel('Nutrition Calculator.xlsx', sheet_name='Nutrition Calculator (p100kcal)',
                         skiprows=(0,1,3), index_col=0)
nut_needs = pd.read_excel('Nutrition Calculator.xlsx', sheet_name='Human Nutritional Needs',
                          nrows=5, skiprows=(0,2), index_col=0)

In [3]:
nut_calc = nut_calc.fillna(0)

In [4]:
nut_needs

,Protein,Fat total,Saturated fats,Trans fats,Carbohydrates,Fiber,Sugars,Vitamin A,Vitamin E,Vitamin D,...,Lysine,Methionine,Cysteine,Phenylalanine,Tyrosine,Threonine,Tryptophan,Valine,Omega-3 (ALA),Omega-6 (LA)
Lower (severe risk),46.00,35.00,1.000000e-99,1.000000e-99,5.000000e+01,2.500000e+01,1.000000e-99,5.000000e+02,1.000000e+04,3.8,...,1.860000e+00,6.200000e-01,2.480000e-01,9.300000e-01,2.480000e-01,9.300000e-01,2.480000e-01,1.612000e+00,1.350000e+00,1.000000e+01
Lower (moderate risk),51.00,46.67,1.000000e-99,1.000000e-99,1.300000e+02,2.500000e+01,1.000000e-99,7.000000e+02,1.500000e+04,15.0,...,1.860000e+00,6.200000e-01,2.480000e-01,9.300000e-01,2.480000e-01,9.300000e-01,2.480000e-01,1.612000e+00,2.000000e+00,1.450000e+01
Higher (safe),78.75,70.00,2.333000e+01,2.300000e+00,4.000000e+02,7.000000e+01,5.250000e+01,3.000000e+03,3.000000e+05,250.0,...,7.440000e+00,2.480000e+00,9.920000e-01,3.720000e+00,9.920000e-01,3.720000e+00,9.920000e-01,6.448000e+00,2.800000e+00,1.867000e+01
Higher (moderate risk),140.00,81.67,2.333000e+01,2.300000e+00,5.000000e+02,5.000000e+02,1.312500e+02,8.000000e+03,1.000000e+06,1925.0,...,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,4.670000e+00,2.333000e+01
Higher (severe risk),400.00,200.00,2.000000e+02,2.000000e+02,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,15000.0,...,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99,1.000000e+99


In [5]:
nut_calc

,Notes,Source,Energy,Protein digestibility,Protein,Fat total,Saturated fats,Trans fats,Carbohydrates,Fiber,...,Lysine,Methionine,Cysteine,Phenylalanine,Tyrosine,Threonine,Tryptophan,Valine,Omega-3 (ALA),Omega-6 (LA)
Food,,,,,,,,,,,,,,,,,,,,,
"Fish, tuna, fresh, skipjack, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,103.000000,0.90000,21.359223,0.980583,0.318447,0.000000,0.000000,0.000000,...,1.961165,0.632039,0.229126,0.833981,0.721359,0.935922,0.238835,1.100000,0.000000,0.015534
"Beef, ground meat","mechanically separated, raw",https://fdc.nal.usda.gov/fdc-app.html#/food-de...,276.000000,0.95000,5.423913,8.521739,4.268116,0.000000,0.000000,0.000000,...,0.420652,0.154348,0.083333,0.233333,0.138043,0.171739,0.060870,0.331522,0.061594,0.213768
"Beef, variety meats and by-products, liver, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,135.000000,0.95000,15.081481,2.688889,0.913333,0.125926,2.881481,0.000000,...,1.190370,0.402222,0.278519,0.802963,0.597778,0.643704,0.194815,0.933333,0.005185,0.221481
"Chicken, broilers or fryers, meat and skin and giblets and neck, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,213.000000,1.00000,8.605634,6.962441,1.990610,0.000000,0.061033,0.000000,...,0.690141,0.226291,0.115493,0.337089,0.275117,0.356808,0.095305,0.419249,0.000000,1.328638
"Mushrooms, shiitake, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,34.000000,0.69000,6.588235,1.441176,0.000000,0.000000,19.970588,7.352941,...,0.394118,0.097059,0.064706,0.326471,0.229412,0.394118,0.032353,0.426471,0.000000,0.000000
"Mushrooms, oyster, raw",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,33.000000,0.60305,10.030303,1.242424,0.187879,0.000000,18.454545,6.969697,...,0.381818,0.127273,0.084848,0.339394,0.254545,0.424242,0.127273,0.596970,0.000000,0.372727
"Milk, whole","whole, 3.25% milkfat, without added vitamin A ...",https://fdc.nal.usda.gov/fdc-app.html#/food-de...,61.000000,0.94000,5.163934,5.360656,3.057377,0.000000,7.836066,0.000000,...,0.432787,0.136066,0.031148,0.267213,0.260656,0.219672,0.065574,0.337705,0.122951,0.196721
"Egg, whole, raw, fresh",0,https://fdc.nal.usda.gov/fdc-app.html#/food-de...,143.000000,0.97000,8.783217,6.650350,2.186014,0.026573,0.503497,0.000000,...,0.637762,0.265734,0.190210,0.475524,0.348951,0.388811,0.116783,0.600000,0.025175,1.087413
ArborVitae Tea,0,Vitamins backup sheet,40.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Trying to combine foods to reach avg of adequate intake.
With

`food_subset=['Beef, variety meats and by-products, mechanically separated beef, raw',
             'Milk, whole, 3.25% milkfat, without added vitamin A and vitamin D',
            'Rice, white, long-grain, regular, raw, unenriched',
            'Corn, sweet, yellow, raw','Barley, hulled','Anchovy, canned','Soy flour, full-fat, raw',
            'Soybeans, mature seeds, raw', 'Beef, organ meal, cooked', 'Bread, whole-wheat']
`

We should find something like `[100, 100, 250, 300, 200, 250, 400, 150, 100, 250]`

In [ ]:
def f(x, target):
    return (target-x)/target

def weighted_errors(val, nut_needs_subset, nut, target):
    lower_severe = nut_needs_subset.at['Lower (severe risk)', nut]
    lower_moderate = nut_needs_subset.at['Lower (moderate risk)', nut]
    higher_safe = nut_needs_subset.at['Higher (safe)', nut]
    higher_moderate = nut_needs_subset.at['Higher (moderate risk)', nut]
    higher_severe = nut_needs_subset.at['Higher (severe risk)', nut]
    if val>higher_severe:
        y = np.exp(f(val, target)) - np.exp(f(higher_severe, target)) + f(higher_severe, target)**4 - f(higher_moderate, target)**4 + f(higher_moderate, target)**2 - f(higher_safe, target)**2
    elif val>higher_moderate:
        y = f(val, target)**4 - f(higher_moderate, target)**4 + f(higher_moderate, target)**2 - f(higher_safe, target)**2
    elif val>higher_safe:
        y = f(val, target)**2 - f(higher_safe, target)**2
    elif (val>= lower_moderate) and (val<=higher_safe):
        y = 0
    elif val<lower_severe:
        y = f(val, target)**4 - f(lower_severe, target)**4 + f(lower_severe, target)**2 - f(lower_moderate, target)**2
    elif val<lower_moderate:
        y = f(val, target)**2 - f(lower_moderate, target)**2
    return y

def intake_cat(X, nut_needs, nut_subset, nut_calc, food_subset):
    nut_calc_subset=nut_calc.loc[food_subset]
    nut_calc_subset=np.array(nut_calc_subset[nut_subset])
    nut_needs_subset = nut_needs[nut_subset]
    #target = 1.1*np.array(nut_needs_subset.loc['Lower (moderate risk)']
    target = (np.array(nut_needs_subset.loc['Lower (moderate risk)'])+np.array(nut_needs_subset.loc['Higher (safe)']))/2
    #print(X)
    solution = np.abs(np.dot(np.transpose(nut_calc_subset), X)-target)
    Y=np.zeros((len(nut_subset)))
    for i in range(len(nut_subset)):
        nut = nut_subset[i]
        val = solution[i]
        Y[i] = weighted_errors(val, nut_needs_subset, nut, target[i])
        #print(nut, val, Y[i])
    #print(np.sum(Y))
    return np.sum(Y)

In [ ]:
'''food_subset=['Beef, variety meats and by-products, mechanically separated beef, raw',
             'Milk, whole, 3.25% milkfat, without added vitamin A and vitamin D',
            'Rice, white, long-grain, regular, raw, unenriched',
            'Corn, sweet, yellow, raw','Barley, hulled','Anchovy, canned','Soy flour, full-fat, raw',
            'Soybeans, mature seeds, raw', 'Beef, organ meal, cooked', 'Bread, whole-wheat']
'''
food_subset = ['Beef, variety meats and by-products, mechanically separated beef, raw',
               'Milk, whole, 3.25% milkfat, without added vitamin A and vitamin D',
               'Potatoes', 'Fish, anchovy, european, raw', 'Beef, organ meal, cooked',]

#nut_subset=['Protein', 'Fat total']
#nut_subset=['Protein', 'Fat total', 'Carbohydrates']
#nut_subset=nut_needs.columns
nut_subset=['Protein', 'Fat total',
            #'Saturated fats', 'Trans fats',
            'Carbohydrates','Fiber',
            #'Sugars',
            'Vitamin A', 'Vitamin E', 'Vitamin D', 'Vitamin C',
           'Thiamine (B1)', 'Riboflavin (B2)', 'Niacin (B3)', 'Vitamin B6',
           'Vitamin B12', 'Vitamin K', 'Folate (B9)', 'Pantothenic Acid (B5)',
           'Calcium', 'Iron', 'Magnesium', 'Phosphorus', 'Potassium', 'Sodium',
           'Zinc', 'Copper', 'Manganese', 'Selenium', 'Iodine', 'Omega-3 (ALA)', 'Omega-6 (LA)']


In [ ]:
bounds = optimize.Bounds(np.zeros(len(food_subset)), 21*np.ones(len(food_subset)))
constraint = optimize.LinearConstraint(np.ones(len(food_subset)), 21, 21)

coef_list = []
for i in range(1):
    X0 = 10*np.random.rand(len(food_subset), 1)
    #X0 = np.ones(len(food_subset))
    coefs = optimize.minimize(intake_cat, X0,
                              (nut_needs, nut_subset, nut_calc, food_subset),
                              constraints=constraint,
                              bounds=bounds,
                             method='SLSQP')
    if coefs.success:
        coef_list += [[coefs.x, coefs.fun]]
        print(i)
        for j in range(len(food_subset)):
            print(food_subset[j][:12], '...:', round(coefs.x[j]*100), 'kcal')
coef_array = np.array(coef_list)
i_min = np.argmin(coef_array[:, 1])
selected_coef = coef_array[i_min][0]
print('Min of 100 trials')
for i in range(len(food_subset)):
    print(food_subset[i][:12], '...:', round(selected_coef[i]*100), 'kcal')

In [ ]:
nut_calc_subset=nut_calc.loc[food_subset]
nut_calc_subset_t=nut_calc_subset[nut_needs.columns].T
combination = pd.Series(selected_coef, index=food_subset)
#combination = pd.Series([100, 100, 250, 300, 200, 250, 400, 150, 100, 250], index=food_subset)/100
diet = nut_calc_subset_t @ combination
print(diet)

nut_needs_subset = nut_needs[nut_subset]
SRAI = []
MRAI = []
AI = []
danger = []
for i in range(len(diet)):
    nutrient=diet[i]
    nut=diet.index[i]
    lower_severe = nut_needs.at['Lower (severe risk)', nut]
    lower_moderate = nut_needs.at['Lower (moderate risk)', nut]
    higher_safe = nut_needs.at['Higher (safe)', nut]
    higher_moderate = nut_needs.at['Higher (moderate risk)', nut]
    higher_severe = nut_needs.at['Higher (severe risk)', nut]
    if (nutrient>= lower_moderate) and (nutrient<=higher_safe):
        AI += [nut]
    elif (nutrient==0):
        danger+= [nut + ' 0%']
    elif (nutrient>higher_severe):
        danger+= [nut + ' ' + str(round(nutrient/higher_safe*100))+'%']
    elif (nutrient<lower_severe):
        SRAI += [nut + ' ' + str(round(nutrient/lower_moderate*100))+'%']
    elif (nutrient>higher_moderate):
        SRAI += [nut + ' ' + str(round(nutrient/higher_safe*100))+'%']
    elif (nutrient< lower_moderate):
        MRAI += [nut + ' ' + str(round(nutrient/lower_moderate*100))+'%']
    elif (nutrient>higher_safe):
        MRAI += [nut + ' ' + str(round(nutrient/higher_safe*100))+'%']
        
print('AI', AI, '\n\nMRAI', MRAI, '\n\nSRAI', SRAI, '\n\nDanger', danger)

In [ ]:
nut_subset_fig=['Fat total']

nut_calc_subset=nut_calc.loc[food_subset]
nut_calc_subset=nut_calc_subset[nut_subset_fig]
nut_needs_subset = nut_needs[nut_subset_fig]
nut = nut_needs_subset.columns[0]

lower_severe = nut_needs_subset.at['Lower (severe risk)', nut]
lower_moderate = nut_needs_subset.at['Lower (moderate risk)', nut]
higher_safe = nut_needs_subset.at['Higher (safe)', nut]
higher_moderate = nut_needs_subset.at['Higher (moderate risk)', nut]
higher_severe = nut_needs_subset.at['Higher (severe risk)', nut]

target = (np.array(nut_needs_subset.loc['Lower (moderate risk)'])+np.array(nut_needs_subset.loc['Higher (safe)']))/2

X = np.linspace(0, higher_moderate+10)
Y=np.zeros(X.shape)

for i in range(len(X)):
    val=X[i]
    Y[i] = weighted_errors(val, nut_needs_subset, nut, target)   
    #print(val, err, Y[i])
        
plt.plot(X, Y)
plt.axvline(0, color='r')
plt.axvline(lower_severe, color='orange')
plt.axvline(lower_moderate, color='green')
plt.axvline(higher_safe, color='lightgreen')
plt.axvline(higher_moderate, color='peachpuff')
#plt.axvline(higher_severe, color='lightcoral')
plt.title(nut_subset_fig[0] + ' weighted error function')